In [1]:
import setup  # notebooks/setup.py
setup.init()

In [2]:
from mypermit import permit_client

permit = permit_client

In [3]:
await permit.api.users.list()

PaginatedResultUserRead(data=[UserRead(key='2', id=UUID('091c69a7-172a-4cfd-ae2e-838dbd512def'), organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a'), project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6'), environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed'), associated_tenants=[], roles=[], created_at=datetime.datetime(2025, 6, 4, 17, 29, 55, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 6, 4, 17, 31, 4, tzinfo=datetime.timezone.utc), email=None, first_name='staff', last_name=None, attributes=None), UserRead(key='3', id=UUID('68ae3c77-0d73-4346-b214-dda04908fcb6'), organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a'), project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6'), environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed'), associated_tenants=[], roles=[], created_at=datetime.datetime(2025, 6, 4, 17, 29, 55, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 6, 4, 17, 31, 5, tzinfo=datetime.timezone.utc), email=None

In [4]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [5]:
for u in User.objects.all():
    user_data = {
        "key": f"{u.id}",
        "first_name": f"{u.username}"
    }
    response = await permit.api.users.sync(user_data)
    print(response)

key='1' id=UUID('f44a4c77-8d9b-488d-b02a-44ecf6bc944a') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6') environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed') associated_tenants=[UserInTenant(tenant='default', roles=['viewer'], status=<UserStatus.active: 'active'>, resource_instance_roles=[])] roles=[UserRole(role='viewer', tenant='default')] created_at=datetime.datetime(2025, 6, 4, 17, 29, 54, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 6, 4, 17, 31, 4, tzinfo=datetime.timezone.utc) email=None first_name='cfe' last_name=None attributes=None
key='2' id=UUID('091c69a7-172a-4cfd-ae2e-838dbd512def') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6') environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed') associated_tenants=[] roles=[] created_at=datetime.datetime(2025, 6, 4, 17, 29, 55, tzinfo=datetime.timezone.utc) updated_at

In [6]:
await permit.check("1", "read", "document")

True

In [7]:
resources = [
    {
        "name": "Document",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {},
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_disovery",
        "actions": {
            "search": {},
            "detail": {},
        }
    }
]

In [8]:
from permit import PermitApiError

async def get_or_create_resource(resource_data:dict):
    resource_key = resource_data.get('key')
    try:
        _resource = await permit.api.resources.get(resource_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.resources.create(resource_data)
        else:
            raise e
    return _resource

In [9]:
for resource_data in resources:
    response = await get_or_create_resource(resource_data)
    print(response)

key='document' id=UUID('3f71f9fd-14f4-4702-88f2-6745a0b68003') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6') environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed') created_at=datetime.datetime(2025, 6, 4, 17, 40, 6, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 6, 4, 17, 40, 6, tzinfo=datetime.timezone.utc) name='Document' urn='prn:hungrypy:default:document' description=None actions={'delete': ActionBlockRead(name='delete', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('83b57a4d-1661-4923-9122-80828b84b98f'), key='delete'), 'update': ActionBlockRead(name='update', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('6db441b0-bda1-46f5-adc7-ca5121d157d9'), key='update'), 'create': ActionBlockRead(name='create', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('71112134-a426-4ed0-b2f9-764f5fc94

In [10]:
# readonly_permissions = [
#     "document:read",
#     "movie_disovery:search",
#     "movie_disovery:detail"
# ]

In [11]:
roles = [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": [
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }, 
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions": [
            "document:read",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }
]

In [12]:
from permit import PermitApiError

async def get_or_create_role(role_data:dict):
    _key = role_data.get('key')
    try:
        _resource = await permit.api.roles.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.roles.create(role_data)
        else:
            raise e
    return _resource

In [13]:
# await permit.api.roles.delete("viewer")

In [14]:
for role_data in roles:
    response = await get_or_create_role(role_data)
    print(response)

name='Manager' description=None permissions=['movie_disovery:detail', 'document:create', 'movie_disovery:search', 'document:update', 'document:read'] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='manager' id=UUID('99f5638e-c457-408b-bb54-c4167b228d17') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6') environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed') created_at=datetime.datetime(2025, 6, 4, 17, 47, 18, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 6, 4, 17, 47, 18, tzinfo=datetime.timezone.utc)
name='Viewer' description=None permissions=['document:read', 'movie_disovery:detail', 'movie_disovery:search'] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='viewer' id=UUID('e6f8048b-3724-4b78-93ce-91069b723ac6') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425

In [15]:
user = User.objects.all().first()

In [46]:
user.id

1

In [50]:
response = await permit.api.users.assign_role({
    "user": f"{user.id}",
    "role": "admin",
    "tenant": "default"
})
response

RoleAssignmentRead(id=UUID('1065f938-eb46-4798-a51f-988d57e106b3'), user='1', role='admin', tenant='default', resource_instance=None, resource_instance_id=None, user_id=UUID('f44a4c77-8d9b-488d-b02a-44ecf6bc944a'), role_id=UUID('70304e6f-7319-4a7e-ba9c-c7ddbdf2ddec'), tenant_id=UUID('f99e0569-2824-4fe5-a820-43e804eb5d47'), organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a'), project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6'), environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed'), created_at=datetime.datetime(2025, 6, 4, 18, 19, 35, tzinfo=datetime.timezone.utc))

In [51]:
await permit.check(f"{user.id}", "delete", "document")

True

In [52]:
from langgraph.checkpoint.memory import InMemorySaver 
from ai.supervisors import get_supervisor
import uuid 

checkpointer = InMemorySaver()
supervisor = get_supervisor(model="gpt-4o", checkpointer=checkpointer)

In [53]:
doc_id = 24
user_id = user.id
user_id

1

In [54]:

config = {"configurable": {"user_id": user_id, "thread_id": uuid.uuid4()}}

response = supervisor.invoke(
    {"messages": 
        [
            {"role": "user", "content": "what are my recent documents?"}
        ]
    },
    config
)

In [55]:
response['messages'][-1].content

"I've provided the list of your recent documents. If you need further assistance with them, feel free to ask!"

In [57]:
response = supervisor.invoke(
    {"messages": 
        [
            {"role": "user", "content": f"Delete any random document by id"}
        ]
    },
    config
)
response['messages'][-1].content

'A random document with ID 16 has been successfully deleted. If you need any more help, feel free to ask!'